# _Influencia de la duración de piststops en carreras de F1_

* Hipótesis:  

- Mejor media de tiempos por carrera y temporada aumenta las posiciones en carrera y al final de temporada  

- Idem para constructores  

- Diferencias entre conductores de una mismo equipo

<img src="F1_logo.jpg"  width="200"> <p>
<img src="pitStopAstonMartin.jpg"  width="600" height="300">

### <span style="color:RED"> DATASET</span>

#### CSV's EXTRAIDOS DE ergast.com  

* Ofrece resultados de todas las carreras desde 1950 hasta hoy.  

* Paquete ergast_py  
https://pypi.org/project/ergast-py/

* API con datos actualizados tras cada carrera  
  
  * API documentación http://ergast.com/mrd/  
    
* Libreria propia para acceder a la API  
    pyERGAST


#### Ofrecen todo el paquete de csv hasta la carrera más reciente  
* f1db_csv.zip  
http://ergast.com/downloads/f1db_csv.zip  

* Diagrama de la BD  
http://ergast.com/images/ergast_db.png  

* Esquema de la BD  
http://ergast.com/docs/f1db_user_guide.txt  

Cada csv tiene una sola base de datos en UTF-8  
La primera línea son las cabeceras de las columnas. 




In [2]:
import pandas as pd
import numpy as np
import sqlite3
import os
import re

* 13 csv

In [3]:
circuits=pd.read_csv(r'./circuits.csv')
constructor_results=pd.read_csv(r'./constructor_results.csv')
constructor_standings=pd.read_csv(r'./constructor_standings.csv')
driver_standings=pd.read_csv(r'./driver_standings.csv')
drivers=pd.read_csv (r'./drivers.csv')
lap_times=pd.read_csv(r'./lap_times.csv')
pit_stops=pd.read_csv(r'./pit_stops.csv')
qualifying=pd.read_csv(r'./qualifying.csv')
races=pd.read_csv(r'./races.csv')
results=pd.read_csv(r'./results.csv')
seasons=pd.read_csv(r'./seasons.csv')
sprint_results=pd.read_csv(r'./sprint_results.csv')
status=pd.read_csv(r'./status.csv')


In [4]:
status[status.status.str.contains("[Tt]yre")]
# usar regex para filtrar valores de columnas

,statusId,status
26,27,Tyre
58,59,Tyre puncture


In [5]:
status.query('status.str.contains("[Tt]yre")')

,statusId,status
26,27,Tyre
58,59,Tyre puncture


* Para el primer proyecto EDA, los más interesantes son:  

    * resultados                    | *results.csv*  
    * conductores                   |  *drivers.csv*  
    * carreras                      |  *races.csv*  
    * resultados de constructores por campeonato   | *constructor_standings*  
    * resultados de constructores por carrera   | *constructor_results*  
    * pit stops                     | *pit_stops*  
    * status (finalizado/otros)     | *status*


In [6]:
# Resultados de cada carrera, identificadores de carrera, piloto, posición, puntos, vueltas, tiempos

results.sample(5)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
2650,2651,145,44,16,9,11,7,7,7,0.0,65,+1:04.977,5528282,\N,\N,\N,\N,1
11789,11790,491,212,34,6,13,5,5,5,2.0,67,\N,\N,\N,\N,\N,\N,11
13770,13771,561,255,32,6,25,\N,R,22,0.0,18,\N,\N,\N,\N,\N,\N,8
13384,13385,548,246,63,18,19,\N,R,16,0.0,41,\N,\N,\N,\N,\N,\N,69
8310,8311,367,110,35,22,17,\N,R,12,0.0,45,\N,\N,\N,\N,\N,\N,5


In [11]:
results.describe().T

,count,mean,std,min,25%,50%,75%,max
resultId,25980.0,12991.356774,7501.122623,1.0,6495.75,12990.5,19485.25,25985.0
raceId,25980.0,534.496651,301.533201,1.0,294.00,517.0,788.00,1105.0
driverId,25980.0,264.395574,270.961060,1.0,57.00,163.0,363.00,858.0
constructorId,25980.0,48.880908,60.019294,1.0,6.00,25.0,58.00,214.0
grid,25980.0,11.172402,7.237330,0.0,5.00,11.0,17.00,34.0
positionOrder,25980.0,12.863202,7.705185,1.0,6.00,12.0,18.00,39.0
points,25980.0,1.894344,4.198993,0.0,0.00,0.0,2.00,50.0
laps,25980.0,46.034450,29.755358,0.0,22.00,53.0,66.00,200.0
statusId,25980.0,17.500154,26.115854,1.0,1.00,10.0,14.00,141.0


In [7]:
# Información básica de cada conductor, nombre, apellido, id, alias

drivers.head(2)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld


In [31]:
races.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101 entries, 0 to 1100
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   raceId       1101 non-null   int64 
 1   year         1101 non-null   int64 
 2   round        1101 non-null   int64 
 3   circuitId    1101 non-null   int64 
 4   name         1101 non-null   object
 5   date         1101 non-null   object
 6   time         1101 non-null   object
 7   url          1101 non-null   object
 8   fp1_date     1101 non-null   object
 9   fp1_time     1101 non-null   object
 10  fp2_date     1101 non-null   object
 11  fp2_time     1101 non-null   object
 12  fp3_date     1101 non-null   object
 13  fp3_time     1101 non-null   object
 14  quali_date   1101 non-null   object
 15  quali_time   1101 non-null   object
 16  sprint_date  1101 non-null   object
 17  sprint_time  1101 non-null   object
dtypes: int64(4), object(14)
memory usage: 155.0+ KB


In [1]:
# Info básica de cada carrera de 1950-2023, id, nombre

races

NameError: name 'races' is not defined

In [36]:
#raceId a int
races["raceId"] = races["raceId"].astype(int)

In [9]:
# Resultado final por carrera por constructor, con puntos

constructor_results.sample(5)

,constructorResultsId,raceId,constructorId,points,status
11552,16052,1030,6,25.0,\N
5352,5353,441,51,0.0,\N
7891,11357,676,196,1.0,\N
4759,4760,399,32,0.0,\N
3809,3810,336,3,3.0,\N


In [11]:
# Resultados por constructor por carrera, puntos, posición

constructor_standings.sample(5)

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
5392,10362,445,1,68.5,1,1,6
1653,5489,179,20,2.0,7,7,0
7341,22938,565,1,21.0,3,3,1
2202,2475,229,27,11.0,5,5,1
12373,27539,1036,51,2.0,8,8,0


In [12]:
# Paradas en boxes de cada carrera y conductor, con duración en segundos y milisegundos

pit_stops.sample(5)

,raceId,driverId,stop,lap,time,duration,milliseconds
5541,968,836,2,24,17:47:52,22.949,22949
7875,1044,822,1,7,13:27:48,23.398,23398
6805,1010,9,2,28,16:58:10,22.343,22343
8083,1053,842,2,26,15:48:45,32.277,32277
9234,1085,855,1,9,15:18:55,36.444,36444


In [37]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25980 entries, 0 to 25979
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   resultId         25980 non-null  int64  
 1   raceId           25980 non-null  int64  
 2   driverId         25980 non-null  int64  
 3   constructorId    25980 non-null  int64  
 4   number           25980 non-null  object 
 5   grid             25980 non-null  int64  
 6   position         25980 non-null  object 
 7   positionText     25980 non-null  object 
 8   positionOrder    25980 non-null  int64  
 9   points           25980 non-null  float64
 10  laps             25980 non-null  int64  
 11  time             25980 non-null  object 
 12  milliseconds     25980 non-null  object 
 13  fastestLap       25980 non-null  object 
 14  rank             25980 non-null  object 
 15  fastestLapTime   25980 non-null  object 
 16  fastestLapSpeed  25980 non-null  object 
 17  statusId    

In [21]:
lap_times.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [22]:
pd.to_datetime(lap_times["time"], format="%M:%S:%f")

ValueError: time data "1:38.109" doesn't match format "%M:%S:%f", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [41]:
lap_times["time"].info()

In [15]:
# CARRERAS DONDE ALONSO HA SIDO 1º 2º o 3º
alonso_1_2_3=results[["raceId","driverId", "position"]].groupby("driverId").get_group(4)[(results.position=="1")|(results.position=="2")|(results.position=="3")]
alonso_1_2_3

C:\Users\j\AppData\Local\Temp\ipykernel_12980\154912858.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  alonso_1_2_3=results[["raceId","driverId", "position"]].groupby("driverId").get_group(4)[(results.position=="1")|(results.position=="2")|(results.position=="3")]


,raceId,driverId,position
288,32,4,1
308,33,4,1
349,35,4,2
369,36,4,2
390,37,4,1
...,...,...,...
25842,1098,4,3
25862,1099,4,3
25882,1100,4,3
25922,1102,4,3


In [16]:
results.groupby("driverId").size()

driverId
1      317
2      184
3      206
4      365
5      112
      ... 
854     44
855     29
856      8
857      7
858      7
Length: 857, dtype: int64